<a href="https://colab.research.google.com/github/anuragbantu/DL-Assignment-2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [1]:
import os
import zipfile

zip_path = '/content/drive/MyDrive/nature_12K.zip'

In [2]:
extract_path = '/content/iNaturalist'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

Dataset extracted to: /content/iNaturalist


# Data Preprocessing